Importing necessary libraries and define custom functions for log returns, realized volatility, and RMSPE from tutorial notebook

In [8]:
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
import xgboost as xgb
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from datetime import datetime

def log_return(list_stock_prices):
    return np.log(list_stock_prices).diff()

def realized_volatility(series_log_return):
    return np.sqrt(np.sum(series_log_return**2))

def rmspe(y_true, y_pred):
    return np.sqrt(np.mean(np.square((y_true - y_pred) / y_true)))

Loading data

In [2]:
train = pd.read_csv('train.csv')
train['target'] = np.sqrt(train['target'])
book = pd.read_parquet('order_book_feature.parquet')
trades = pd.read_parquet('trades.parquet')
time_id_reference = pd.read_csv('time_id_reference.csv', parse_dates=['date'])

Feature engineering. Calculate 'wap', 'bid_ask_spread', and 'order_imbalance' in the order book. Calculate log returns for trades. 
Aggregate features for both book and trades data and merge with the train data

In [3]:
book['wap'] = (book['bid_price1'] * book['ask_size1'] + book['ask_price1'] * book['bid_size1']) / (book['bid_size1'] + book['ask_size1'])
book['bid_ask_spread'] = book['ask_price1'] - book['bid_price1']
book['order_imbalance'] = (book['bid_size1'] - book['ask_size1']) / (book['bid_size1'] + book['ask_size1'])


trades['price_log_return'] = trades.groupby(['time_id', 'stock_id'])['price'].apply(log_return)
trades = trades.dropna()

features = book.groupby(['time_id', 'stock_id']).agg({
    'wap': 'mean',
    'bid_ask_spread': np.var,
    'order_imbalance': np.var,
}).reset_index()

trades_features = trades.groupby(['time_id', 'stock_id']).agg({
    'price_log_return': [realized_volatility],
    'size': 'sum',
    'order_count': 'sum'
}).reset_index()

trades_features.columns = ['_'.join(col) if col[0] != 'time_id' and col[0] != 'stock_id' else col[0] for col in trades_features.columns.values]

merged_features = pd.merge(features, trades_features, on=['time_id', 'stock_id'], how='left')

train_merged = train.merge(merged_features, on=['stock_id', 'time_id'], how='left')


/var/folders/lz/81hq8_d56pjbxtvpn4bqw7hm0000gn/T/ipykernel_14742/1206922324.py:6: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  trades['price_log_return'] = trades.groupby(['time_id', 'stock_id'])['price'].apply(log_return)


Time-based feature engineering. Extract day of week, week of month, and hour of day. Add a binary indicator for Fed meeting dates

In [4]:
time_id_reference['day_of_week'] = time_id_reference['date'].dt.dayofweek
time_id_reference['week_of_month'] = time_id_reference['date'].apply(lambda x: (x.day - 1) // 7 + 1)
time_id_reference['hour_of_day'] = time_id_reference['time'].apply(lambda x: datetime.strptime(x, '%H:%M:%S').hour)

# Add Fed meeting dates
fed_meeting_dates = [
    '2021-01-26', '2021-01-27',
    '2021-03-16', '2021-03-17',
    '2021-04-27', '2021-04-28',
    '2021-06-15', '2021-06-16',
    '2021-07-27', '2021-07-28',
    '2021-09-21', '2021-09-22'
]

time_id_reference['fed_meeting'] = time_id_reference['date'].isin(fed_meeting_dates).astype(int)

train_merged = train_merged.merge(time_id_reference, on=['time_id'], how='left')



One-hot encoding. One-hot encode 'day_of_week' and 'week_of_month' features and merge the one-hot encoded features with the train data.
The one-hot encoding of the day of the week and week of the month creates dummy variables, which allows the model to learn patterns related to specific days and weeks without assuming any ordinal relationship between them. The time of the day is already implicitly included in the model through the time_id. By incorporating these time-related features, the model can learn to adjust its predictions based on the time-related patterns present in the data. If the data shows that certain hours, weekdays, or weeks of the month have higher or lower realized volatility on average, the model will learn these relationships and adjust its predictions accordingly.

In [5]:
one_hot_encoder = OneHotEncoder(sparse=False)
day_of_week_one_hot = one_hot_encoder.fit_transform(train_merged[['day_of_week']])
week_of_month_one_hot = one_hot_encoder.fit_transform(train_merged[['week_of_month']])

day_of_week_columns = [f'day_of_week_{i}' for i in range(day_of_week_one_hot.shape[1])]
week_of_month_columns = [f'week_of_month_{i}' for i in range(week_of_month_one_hot.shape[1])]

day_of_week_df = pd.DataFrame(day_of_week_one_hot, columns=day_of_week_columns)
week_of_month_df = pd.DataFrame(week_of_month_one_hot, columns=week_of_month_columns)

train_merged = pd.concat([train_merged, day_of_week_df, week_of_month_df], axis=1)

train_merged = train_merged.drop(['date', 'time', 'day_of_week', 'week_of_month'], axis=1)

/Users/yskakshiyap/.pyenv/versions/3.10.9/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/Users/yskakshiyap/.pyenv/versions/3.10.9/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Preparing the data for the model
- Drop unnecessary columns from the train data
- Assign sample weights based on whether there was a Fed meeting
- Standardize the feature data using StandardScaler
- Split the data into train and test sets

In [6]:

X = train_merged.drop(['target', 'time_id', 'stock_id'], axis=1)
y = train_merged['target']

sample_weights = np.where(train_merged['fed_meeting'] == 1, 3, 1)  # Assign a weight of 3 to Fed meeting dates, and 1 otherwise

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

Model training and evaluation
- Instantiate an XGBoost Regressor model
- Define a parameter grid for hyperparameter tuning
- Use RandomizedSearchCV for hyperparameter tuning with cross-validation
- Train the model using sample weights
- Evaluate the model performance using R2 score and RMSPE

In [7]:
# Fit the model using the sample weights
model = xgb.XGBRegressor()

param_grid = {
    'n_estimators': [100, 300, 500],
    'max_depth': [3, 6, 9],
    'learning_rate': [0.001, 0.01, 0.05, 0.1],
    'min_child_weight': [5, 10, 50, 100],
    'subsample': [0.5, 0.75, 1],
    'colsample_bytree': [0.5, 0.75, 1],
    'gamma': [0, 0.1, 0.2, 0.3]
}


random_cv = RandomizedSearchCV(model, param_distributions=param_grid, n_jobs=-1, cv=3, scoring="r2", n_iter=50, random_state=42)
random_cv.fit(X_train, y_train, sample_weight=sample_weights[:len(X_train)])

best_model = random_cv.best_estimator_
y_pred = best_model.predict(X_test)

R2 = round(r2_score(y_true=y_test, y_pred=y_pred), 3)
RMSPE = round(rmspe(y_true=y_test, y_pred=y_pred), 3)

print(f'Performance of the XGBoost model with feature engineering combining both fed meeting dates and special time_id references: R2 score: {R2}, RMSPE: {RMSPE}')


Performance of the XGBoost model with feature engineering combining both fed meeting dates and special time_id references: R2 score: 0.929, RMSPE: 0.078


In [9]:
mae = mean_absolute_error(y_test, y_pred)
print("Mean Absolute Error (MAE):", mae)

# Calculate Mean Squared Error (MSE)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error (MSE):", mse)

# Calculate Root Mean Squared Error (RMSE)
rmse = np.sqrt(mse)
print("Root Mean Squared Error (RMSE):", rmse)

# Calculate Mean Absolute Percentage Error (MAPE)
mape = mean_absolute_percentage_error(y_test, y_pred)
print("Mean Absolute Percentage Error (MAPE):", mape)

Mean Absolute Error (MAE): 0.0036606616429689623
Mean Squared Error (MSE): 2.580324771988886e-05
Root Mean Squared Error (RMSE): 0.0050796897267341885
Mean Absolute Percentage Error (MAPE): 0.05968371693309086
